### Loading our ResNet model :

In [1]:
from keras.models import load_model

modelpath='../ResNet92PercentVer2_1.h5'
model= load_model(modelpath)


### Creating the live feed , with openCV.
### Each time it recognizes a hand-object , waits for couple of runs to check for stability , once the required stability is achieved, we send the associated frame of the video to the model to evaluate.

In [86]:
import cv2
import numpy as np

camera = cv2.VideoCapture(0)
prev_ContursLength=0
stable_Count = 0
capture_Count = 0
prev_Prediction = 0
while camera.isOpened():
    ret , frame=camera.read()
    k=cv2.waitKey(10)
    if k==32:
        break
    # frame of a detectable object
    iou=frame[170:280, 170:280]
    bound_box=cv2.rectangle(frame,(100,100),(340,340),(255,0,0),1)
    rgbIoU=cv2.cvtColor(iou,cv2.COLOR_BGR2RGB)
    # the captures we will feed the model with
    snapShot=frame[101:340,101:340]
    # skin color filtering - to detect hands better than other objects
    lower_skin = np.array([30,20,0], dtype=np.uint8)
    upper_skin = np.array([240,220,210], dtype=np.uint8)
    # mask and threshold for better hand-filtering
    maskBGR= cv2.inRange(rgbIoU,lower_skin,upper_skin)
    thresh = cv2.threshold(maskBGR, 40, 255, cv2.THRESH_OTSU)[1]
    thresh = cv2.erode(thresh, None, iterations=3)
    thresh = cv2.dilate(thresh, None, iterations=3)
    hand_conturs=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    maskCheck=np.array([255,255,255],dtype=np.uint8)
    if len(hand_conturs)>3:
            stable_Count+=1
            if(stable_Count>16):
                capture_Count+=1
                stable_Count=0
                prediction = predict_Gesture(snapShot,model)
                if(prev_Prediction != prediction):
                    prev_Prediction = prediction

            cv2.putText(frame,str(prediction),(120,50), font, 2, (0,202,0), 2,cv2.LINE_AA)
            cv2.drawContours(frame,hand_conturs,-1,(0,250,0), 1)
            anchor_box=cv2.rectangle(frame,(170,170),(280,280),(0,255,0),1)
            cv2.rectangle(frame,(100,100),(340,340),(0,255,0),1)
    else:
        #initialize stable count and prediction value
        prediction=""
        stable_Count=0

        #show the window
    cv2.imshow('frame',frame)
    
camera.release()
cv2.destroyAllWindows()


In [2]:
# Function that fits the image to the model, and returns the prediction
from keras.models import load_model
from matplotlib.pyplot import imshow,imread
import scipy.misc
import numpy as np
from keras.preprocessing import image
from matplotlib.pyplot import imshow,imread
import cv2
def predict_Gesture(image_feed,model):
    x = cv2.resize(image_feed,(64,64))
    x = np.expand_dims(x, axis=0)
    x = x/255.0
    prediction = model.predict(x)
    print(prediction)
    print("Highest value:", scipy.amax(prediction),", Classified as: ", scipy.argmax(prediction))
    return scipy.argmax(prediction)
